# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [1]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
from tqdm import tqdm
import vllm

import genparse
from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [9]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

In [2]:
detailed_debug_sentences_path = Path().resolve() / "debug_sentences_detailed.jsonl"
detailed_debug_sentences = [json.loads(line) for line in detailed_debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [10]:
sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
def show_sentences(sentence_data):
    lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
    for sentence_datum in sentence_data:
        lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
    markdown = "\n".join(lines)
    display(Markdown(markdown))


show_sentences(debug_sentences)

| Sentence | Example PClean preamble code |
| --- | --- |
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "neurologist"</pre> |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "cardiologist"</pre> |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "pediatrician"</pre> |

## Helper code

### SpaCy code

In [11]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [12]:
def extract_code_from_response(text: str) -> str:
    result = text.removeprefix('<|start_header_id|>assistant<|end_header_id|>').strip()
    return result

In [13]:
# Test to confirm it works as intended
text = '''<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```'''
extract_code_from_response(text)

'Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:\n\n```julia\nname_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)\nrow_trace[name_key] = "Kay Ryan"\ncity_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)\nrow_trace[city_key] = "Baltimore"\n```'

### Sort posterior

In [14]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [15]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [16]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running inference locally

In [17]:
SERVER_MODEL_ID = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
SERVER_GENPARSE_MODEL_NAME = "llama3.1"
_LOCAL_BATCH_SIZE = 1

class FakeResponse:
    def __init__(self, data):
        self._data = data
    
    @property
    def status_code(self):
        return 200

    def json(self):
        return self._data

    def text(self):
        return json.dumps(self._data)

def server_model(grammar, proposal="character"):
    return InferenceSetupVLLM(SERVER_GENPARSE_MODEL_NAME, grammar, proposal_name=proposal)

def run_inference_vllm(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _LOCAL_BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = "",
    inference_setup: InferenceSetupVLLM,
) -> requests.Response:
    #assert batch_size == inference_setup.sampler.llm._model.batch_size
    assert grammar == inference_setup.sampler.grammar
    assert (proposal == 'character' and isinstance(inference_setup, genparse.CharacterProposal))
    inference = inference_setup(prompt, n_particles=n_particles, temperature=temperature, max_tokens=max_tokens)
    return FakeResponse({"log_ml": None, "posterior": inference.posterior})

### Running the server

In [18]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> requests.Response:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

### PClean grammar

In [19]:
pclean_grammar = r"""
start: prefix julia_code suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JULIA? NL
suffix: NL CODE_FENCE
julia_code: add_to_trace (NL+ add_to_trace)+

FREE_TEXT: /[a-zA-Z0-9.,-?!;: ]+/
CODE_FENCE: "```"
JULIA: "julia"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9. ]*"/

add_to_trace: get_key NL set_key_in_trace
# slightly overly restrictive but good enough 
get_key: trace_key_identifier WS* "=" WS* "PClean.resolve_dot_expression(trace.model, :Obs, " column_symbol ")"
# column_symbol: /:[a-z][a-z_]+/
column_symbol: ":" ("name" | "address" | "specialty" | "city")
set_key_in_trace: "row_trace[" trace_key_identifier "]" WS* "=" WS* STRING
trace_key_identifier: /[a-z][a-z_]+/
"""
pclean_parser = lark.Lark(pclean_grammar)

### PClean code template

In [20]:
pclean_template = string.Template(
    """
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
$preamble

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:$N
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)
""".lstrip()
)

PCLEAN_DEFAULT_N_SAMPLES = 100

### PClean generation prompt

In [21]:
pclean_prompt = string.Template(
    """Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "256 Overflow St"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "neurology"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

Input: Dr. Pat Rogers's orthopedics office screwed us! Took our money and Kay gave us three minutes tops. #BaltimoreSucks
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Pat Rogers"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "orthopedics"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

In other words, we query all values given in the sentence: The doctor's name, their office address, their specialty, and the city. If one of these is missing, we do not query on it.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_people_prompt(sentence: str) -> str:
    return pclean_prompt.substitute(sentence=sentence)

### Running PClean

In [22]:
def run_pclean(code: str) -> Any:
    # TODO
    raise NotImplementedError()

### Displaying PClean query results

In [23]:
def display_query_results(query_results: Any) -> None:
    # TODO
    raise NotImplementedError()

### JSON generation prompt

In [24]:
json_prompt = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: ```json
{"first": "John", "last": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "addr": "512 Example Street", "addr2": "Suite 3600", "city_name": "Camp Hill"}
```

Input: Loved visiting Dr. Kay Ryan's neurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: ```json
{"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}
```

Input: Dr. Pat Rogers's office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: ```json
{"first": "Pat", "last": "Rogers", "legal_name": "Soulful Medical Services Inc", "city_name": "Baltimore"}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt(sentence: str) -> str:
    return json_prompt.substitute(sentence=sentence)

print(json_prompt)

### JSON grammar

In [25]:
json_grammar = r"""
start: prefix json suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JSON_TAG? NL
suffix: NL CODE_FENCE
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
kv_pair: COLUMN WS* ":" WS* STRING

FREE_TEXT: /[a-zA-Z0-9.,-?!;:' ]+/
CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]*"/
# dropped for now: "city"
COLUMN: "\"" ("first_name" | "last_name" | "address" | "address2" | "specialty" | "c2z3" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

In [26]:
force_nonempty_json_grammar = r"""
start: prefix json suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JSON_TAG? NL
suffix: NL CODE_FENCE
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
#kv_pair: COLUMN WS* ":" WS* STRING
kv_pair: (NONEMPTY_COLUMN WS* ":" WS* NONEMPTY_STRING) | (EMPTYABLE_COLUMN WS* ":" WS* STRING)

FREE_TEXT: /[a-zA-Z0-9.,-?!;:' ]+/
CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
NONEMPTY_STRING: /"[a-zA-Z0-9-.,'& ]+"/
STRING: /"[a-zA-Z0-9-.,'& ]*"/
NONEMPTY_COLUMN: "\"" ("first" | "last" | "specialty | "addr" | "city" | "zip" | "legal_name") "\""
EMPTYABLE_COLUMN: "\"addr2\""
#COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city" | "zip" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

In [36]:
json_grammar = r"""
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? OPTIONAL_WS kv_pair ("," NL? OPTIONAL_WS kv_pair)* NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS STRING

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city_name" | "zip" | "legal_name") "\""
"""
#json_parser = lark.Lark(json_grammar)

# Demo

Example sentences:

1. Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore
2. Dr. Kay Ryan's office at 384 Oak St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare

In [ ]:
MAX_NEW_TOKENS = 256
N_PARTICLES = 15
N_SAMPLES = 100
server_inference_params = {'max_tokens': MAX_NEW_TOKENS, 'n_particles': N_PARTICLES, 'grammar': pclean_grammar}

while True:
    sentence = input('Give me a sentence involving a (named) physician (ideally with specialty, address, or city name): ')
    if sentence == 'q':
        break

    print(f'Processing sentence `{sentence}`...')
    people = get_people(sentence)
    logger.debug(f'People: {people}')
    
    prompt = format_people_prompt(sentence=sentence)
    logger.debug(f'Prompt: {prompt}')

    try:
        response = run_inference_server(prompt, **server_inference_params)
        data = response.json()
    except json.DecodeError as e:
        print(f'ERROR: response status {response.status_code} - {response.text}. Parse error {e}.')
        continue
    aggregate_likelihoods = get_aggregate_likelihoods(data['posterior'])
    best_code, likelihood = get_best_inference(aggregate_likelihoods)
    full_code = pclean_template.substitute(preamble=best_code, N=N_SAMPLES)
    
    display(Markdown('Code:'))
    display(Markdown(f'```julia\n{full_code}\n```'))
    display(Markdown(f'Likelihood: {100 * likelihood:.2f}%'))
    
    try:
        results = run_pclean(full_code)
        display(Markdown('Query Results:'))
        display_query_results(results)
    except NotImplementedError:
        print("Sorry, we can't actually run queries yet. 😅")

## Debugging

In [4]:
model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

### Running the LM unconstrained

In [5]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [6]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

2024-07-29 16:25:33 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): huggingface.co:443
2024-07-29 16:25:33 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 200 0


INFO 07-29 16:25:33 config.py:1417] Upcasting torch.bfloat16 to torch.float32.
INFO 07-29 16:25:33 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)


2024-07-29 16:25:33 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0
2024-07-29 16:25:33 - DEBUG - filelock -   Attempting to acquire lock 140174491775568 on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/6570668dec1f28456bf8488ef82f02bd4580abbc.lock
2024-07-29 16:25:33 - DEBUG - filelock -   Lock 140174491775568 acquired on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/6570668dec1f28456bf8488ef82f02bd4580abbc.lock
2024-07-29 16:25:33 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "GET /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 55276


tokenizer_config.json:   0%|          | 0.00/55.3k [00:00<?, ?B/s]

2024-07-29 16:25:33 - DEBUG - filelock -   Attempting to release lock 140174491775568 on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/6570668dec1f28456bf8488ef82f02bd4580abbc.lock
2024-07-29 16:25:33 - DEBUG - filelock -   Lock 140174491775568 released on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/6570668dec1f28456bf8488ef82f02bd4580abbc.lock
2024-07-29 16:25:33 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer.json HTTP/11" 200 0
2024-07-29 16:25:33 - DEBUG - filelock -   Attempting to acquire lock 140181801218208 on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/5cc5f00a5b203e90a27a3bd60d1ec393b07971e8.lock
2024-07-29 16:25:33 - DEBUG - filelock -   Lock 140181801218208 acquired on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

2024-07-29 16:26:18 - DEBUG - filelock -   Attempting to release lock 140173842397216 on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/0fd8120f1c6acddc268ebc2583058efaf699a771.lock
2024-07-29 16:26:18 - DEBUG - filelock -   Lock 140173842397216 released on /home/joseph/.cache/huggingface/hub/.locks/models--meta-llama--Meta-Llama-3.1-8B-Instruct/0fd8120f1c6acddc268ebc2583058efaf699a771.lock
2024-07-29 16:26:18 - DEBUG - filelock -   Attempting to release lock 140173787725488 on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-29 16:26:18 - DEBUG - filelock -   Lock 140173787725488 released on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-29 16:26:28 model_runner.py:692] Loading model weights took 29.9776 GB
INFO 07-29 16:26:33 gpu_executor.py:102] # GPU blocks: 725, # CPU blocks: 1024
INFO 07-29 16:26:34 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-29 16:26:34 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-29 16:26:58 model_runner.py:1181] Graph capturing finished in 24 secs.


2024-07-29 16:26:59 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): stats.vllm.ai:443
2024-07-29 16:26:59 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

2024-07-29 16:26:58 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0


In [33]:
json_parser = lark.Lark(json_grammar)
json_parser.parse('''<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}''')

Tree(Token('RULE', 'start'), [Token('NL', '\n'), Token('NL', '\n'), Tree(Token('RULE', 'json'), [Token('NL', '\n'), Token('WS', ' '), Token('WS', ' '), Tree(Token('RULE', 'kv_pair'), [Token('COLUMN', '"first"'), Token('WS', ' '), Token('STRING', '"Kay"')]), Token('NL', '\n'), Token('WS', ' '), Token('WS', ' '), Tree(Token('RULE', 'kv_pair'), [Token('COLUMN', '"last"'), Token('WS', ' '), Token('STRING', '"Ryan"')]), Token('NL', '\n'), Token('WS', ' '), Token('WS', ' '), Tree(Token('RULE', 'kv_pair'), [Token('COLUMN', '"addr"'), Token('WS', ' '), Token('STRING', '"321 Pine St."')]), Token('NL', '\n'), Token('WS', ' '), Token('WS', ' '), Tree(Token('RULE', 'kv_pair'), [Token('COLUMN', '"city_name"'), Token('WS', ' '), Token('STRING', '"Baltimore"')]), Token('NL', '\n')])])

In [34]:
sentence = debug_sentences[1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
Prompt (615 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (addit

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.16s/it, est. speed input: 194.84 toks/s, output: 332.01 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St",
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St",
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St",
  "city_name": "Baltimore"
}```
Generated Que

In [29]:
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    for i, output in enumerate(response.outputs, start=1):
        print(f'Generated Query {i}: ```{output.text}```')

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
Prompt (615 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street addre

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.47s/it, est. speed input: 177.42 toks/s, output: 253.58 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.18s/it, est. speed input: 193.71 toks/s, output: 344.89 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St.",
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St",
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St",
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "addr": "321 Pine St",
  "city_name": "Baltimore"
}```
Generated Que

...

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Que

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.19s/it, est. speed input: 192.50 toks/s, output: 268.06 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "city_name": "Baltimore"
}```
Generated Query 8: ```<|s

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.72s/it, est. speed input: 226.37 toks/s, output: 376.29 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrician", 
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay", 
  "last": "Ryan", 
  "specialty": "pediatrici

In [ ]:
for sentence_datum in detailed_debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    for i, output in enumerate(response.outputs, start=1):
        print(f'Generated Query {i}: ```{output.text}```')

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [ ]:
try:
    tokenizer
except NameError:
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [37]:
sentence = debug_sentences[1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

2024-07-29 18:29:02 - DEBUG - urllib3.connectionpool -   Starting new HTTP connection (1): 34.122.30.137:8888


Prompt (615 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please ge

2024-07-29 18:29:14 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 871


Generated Query 1 (likelihood 56.58%): ```<|start_header_id|>assistant<|end_header_id|>
{
    "first": "Kay",
    "last": "Ryan",
    "city_name": "Baltimore",
    "addr": "321 Pine St."
}```
Generated Query 2 (likelihood 16.98%): ```<|start_header_id|>assistant<|end_header_id|>
{
    "first": "Kay",
    "last": "Ryan",
    "legal_name" : "CityCare",
    "addr": "321 Pine St.",
    "city_name": "Baltimore"
}```
Generated Query 3 (likelihood 14.14%): ```<|start_header_id|>assistant<|end_header_id|>
{"city_name": "Baltimore", "first": "Kay", "last": "Ryan", "legal_name": "CityCare", "addr": "321 Pine St."}```
Generated Query 4 (likelihood 12.30%): ```<|start_header_id|>assistant<|end_header_id|>
{"first": "Kay", "last": "Ryan", "legal_name": "City Care Inc", "city_name": "Baltimore", "addr": "321 Pine St."}```


In [38]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    print(response)

2024-07-29 18:31:05 - DEBUG - notebook -   Prompt (615 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et ce

<Response [200]>


2024-07-29 18:31:21 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1556
2024-07-29 18:31:21 - DEBUG - notebook -   Prompt (617 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" 

<Response [200]>


2024-07-29 18:31:27 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1555
2024-07-29 18:31:27 - DEBUG - notebook -   Prompt (617 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" 

...

<Response [200]>


2024-07-29 18:32:07 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1086
2024-07-29 18:32:07 - DEBUG - notebook -   Prompt (616 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's first name)
- "last" (the doctor's last name)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" 

<Response [200]>


2024-07-29 18:32:14 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1665


<Response [200]>


In [39]:
next(iter(responses.values())).content

b'{"log_ml_estimate":-116.66421976627137,"posterior":{"<|start_header_id|>assistant<|end_header_id|>\\n{\\n    \\"first\\": \\"Kay\\",\\n    \\"last\\": \\"Ryan\\",\\n    \\"last\\": \\"doctor\\",\\n    \\"city_name\\": \\"Baltimore\\"\\n}":0.13191638853740031,"<|start_header_id|>assistant<|end_header_id|>\\n{\\n    \\"first\\": \\"Kay\\",\\n    \\"last\\": \\"Ryan\\",\\n    \\"legal_name\\": \\"Baltimore\\",\\n    \\"city_name\\": \\"Baltimore\\",\\n    \\"specialty\\": \\"Baltimore\\",\\n    \\"addr\\": \\"Health Matters\\",\\n    \\"zip\\": \\"Manhatten\\",\\n    \\"addr2\\": \\"\\"\\n}":0.009306235959869453,"<|start_header_id|>assistant<|end_header_id|>\\n{\\n    \\"first\\": \\"Kay\\",\\n    \\"last\\": \\"Ryan\\",\\n    \\"legal_name\\": \\"Baltimore\\",\\n    \\"city_name\\": \\"Baltimore\\",\\n    \\"specialty\\": \\"Baltimore\\",\\n    \\"addr\\": \\"Health Matters\\",\\n    \\"zip\\": \\"Manhatten\\",\\n    \\"addr2\\": \\"Succesfully\\"\\n}":0.009223391635839212,"<|start_hea

In [40]:
def md_html_output_text(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |
|:--- |:--- |:--- | ---:|
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>{<br>    "first": "Kay",    "last": "Ryan",    "city_name": "Baltimore"<br>}</pre> | (unk) | 39.57% (39.57%) |
|     | <pre>{<br>"first": "Kay",<br>"last": "Ryan",<br>"specialty": "neurology",<br>"legal_name": "Green Medicine Inc",<br>"city_name": "Baltimore"<br>}</pre> | (unk) | 26.38% (26.38%) |
|     | <pre>{<br>"first": "Kay",<br>"last": "Ryan",<br>"specialty": "neurology",<br>"legal_name": "Green Medicine Inc",<br>"addr": "256 Overflow St",<br>"zip": "170112202",<br>"city_name": "Camp Hill"<br>}</pre> | (unk) | 13.19% (13.19%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "last": "doctor",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 13.19% (13.19%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "general",<br>    "legal_name": "Happy Brain Services LLC",<br>    "addr": "512 Example Street",<br>    "city_name": "Baltimore",<br>    "zip": "170112202"<br>}</pre> | (unk) | 3.52% (3.52%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Soulful Medical Services Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 1.17% (1.17%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Soulful Medical Services Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 1.07% (1.07%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Baltimore",<br>    "city_name": "Baltimore",<br>    "specialty": "Baltimore",<br>    "addr": "Health Matters",<br>    "zip": "Manhatten",<br>    "addr2": ""<br>}</pre> | (unk) | 0.93% (0.93%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Baltimore",<br>    "city_name": "Baltimore",<br>    "specialty": "Baltimore",<br>    "addr": "Health Matters",<br>    "zip": "Manhatten",<br>    "addr2": "Succesfully"<br>}</pre> | (unk) | 0.92% (0.92%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Soulful Medical Services Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill",<br>    "last": "Monca",<br>    "specialty": "neurology",<br>    "legal_name": "Happy Brain Services LLC",<br>    "addr": "512 Example Street",<br>    "addr2": "Suite 3600"<br>}</pre> | (unk) | 0.04% (0.04%) |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>{"first": "Kay", "last": "Ryan", "addr": "321 Pine St", "city_name": "Baltimore"}</pre> | (unk) | 48.75% (48.75%) |
|     | <pre>{"last": "Ryan", "first": "Kay", "addr": "321 Pine St."}</pre> | (unk) | 36.56% (36.56%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "undefined",<br>    "addr": "321 Pine St.",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 12.19% (12.19%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Green Medicine Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 1.73% (1.73%) |
|     | <pre>{<br>    "specialty": "neurology",<br>    "legal_name": "Happy Brain Services LLC",<br>    "city_name": "Camp Hill",<br>    "addr": "512 Example Street",<br>    "addr2": "Suite 3600",<br>    "last": "Smith",<br>    "first": "John"<br>}</pre> | (unk) | 0.56% (0.56%) |
|     | <pre>{<br>    "specialty": "neurology",<br>    "legal_name": "Happy Brain Services LLC",<br>    "city_name": "Camp Hill",<br>    "addr": "512 Example Street",<br>    "addr2": "Suite 3600",<br>    "last": "Smith",<br>    "first": "John",<br>    "zip": "170112202"<br>}</pre> | (unk) | 0.21% (0.21%) |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>{"first": "Kay", "last": "Ryan", "legal_name": "", "city_name": "Baltimore"}</pre> | (unk) | 86.38% (86.38%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Green Medicine Inc",<br>    "specialty": "neurology",<br>    "addr2": "256",<br>    "addr": "Overflow St",<br>    "zip": "17011-2202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 10.44% (10.44%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Green Medicine Inc",<br>    "specialty": "neurology",<br>    "addr2": "256",<br>    "addr": "Overflow St",<br>    "zip": "17011-2202",<br>    "city_name": "CampHill"<br>}</pre> | (unk) | 1.84% (1.84%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "oncologist",<br>    "legal_name": "Artful Medical Services Inc",<br>    "addr": "000 Art Street",<br>    "addr2": "Suite 4007",<br>    "city_name": "Baltimore",<br>    "zip": "941512202"<br>}</pre> | (unk) | 0.68% (0.68%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "oncologist",<br>    "legal_name": "Artful Medical Services Inc",<br>    "addr": "000 Art Street",<br>    "addr2": "Suite 4007",<br>    "city_name": "Baltimore",<br>    "zip": "941533510"<br>}</pre> | (unk) | 0.66% (0.66%) |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurologist", "city_name": "Baltimore"}</pre> | (unk) | 68.62% (68.62%) |
|     | <pre>{"last": "Ryan", "specialty": "neurology", "city_name": "Baltimore"}</pre> | (unk) | 16.74% (16.74%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "zip": "21122", "specialty": "neurologist"}</pre> | (unk) | 2.44% (2.44%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "zip": "21122", "specialty": "neurologist", "addr": "512 Example Street Suite 3600", "legal_name": "Green Medicine Inc"}</pre> | (unk) | 2.42% (2.42%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "zip": "21122", "specialty": "neurologist", "legal_name": "Healthcare"}</pre> | (unk) | 2.20% (2.20%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "zip": "21122", "specialty": "neurology", "legal_name": "Ryan Neurologists Med Group LLC"}</pre> | (unk) | 2.04% (2.04%) |
|     | <pre>{"last": "Ryan", "specialty": "neurologist", "legal_name": "excellent", "city_name": "Baltimore", "addr": "above this line", "zip": "below this line"}</pre> | (unk) | 1.95% (1.95%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "zip": "21122", "specialty": "neurologist", "legal_name": "Happy Brain Services LLC"}</pre> | (unk) | 1.92% (1.92%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "zip": "21122", "specialty": "neurologist", "legal_name": "Ryan Medical", "addr": "1234 Provider St"}</pre> | (unk) | 1.03% (1.03%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "legal_name": "Grateful for her expertise", "specialty": "neurologist"}</pre> | (unk) | 0.40% (0.40%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore", "legal_name": "Grateful for her expertise", "addr": "neurologist", "specialty": "Dr."}</pre> | (unk) | 0.25% (0.25%) |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}</pre> | (unk) | 49.46% (49.46%) |
|     | <pre>{"first": "Kay", "last": "Ryan"}</pre> | (unk) | 41.21% (41.21%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Baltimore"}</pre> | (unk) | 8.24% (8.24%) |
|     | <pre>{"specialty": "family medicine", "legal_name": "Trusted Care Inc.", "first": "Kay", "last": "Ryan", "city_name": "Baltimore", "addr": "911 Slainti Beal Dhicut Moileamhas Bhn", "addr2": "7200"}</pre> | (unk) | 0.57% (0.57%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "neurology", "legal_name": "City of Charm", "city_name": "Baltimore", "addr": "City Hall 100 N Holliday St", "zip": "21202"}</pre> | (unk) | 0.51% (0.51%) |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore",<br>    "zip": "21001-0200"<br>}</pre> | (unk) | 25.96% (25.96%) |
|     | <pre>{<br>"first": "Kay", "last": "Ryan", "specialty": "physical therapy", "legal_name": "Distinct Medical Services Inc", "addr": "321 Pine St", "zip": "17221520", "city_name": "Baltimore"<br>}</pre> | (unk) | 17.31% (17.31%) |
|     | <pre>{<br>"first": "Kay", "last": "Ryan", "specialty": "physical therapy", "legal_name": "Distinct Medical Services Inc", "addr": "321 Pine St", "city_name": "Baltimore"<br>}</pre> | (unk) | 17.31% (17.31%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Health Check",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore",<br>    "specialty": "assistant"<br>}</pre> | (unk) | 17.31% (17.31%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "legal_name": "Health Check",<br>    "addr": "321 Pine St",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 17.31% (17.31%) |
|     | <pre>{<br>"first": "Kay", "last": "Ryan", "specialty": "physical therapy", "legal_name": "Distinct Medical Services Inc", "addr": "321 Pine St", "zip": "33114-0250", "city_name": "Baltimore"<br>}</pre> | (unk) | 3.64% (3.64%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Busy Brain Clinic Inc.",<br>    "addr": "321 Pine St",<br>    "zip": "111111111",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 1.16% (1.16%) |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>{"first": "Ryan", "last": "Kay", "specialty": "cardiology", "city_name": "Baltimore"}</pre> | (unk) | 39.01% (39.01%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "cardiology", "addr": "256 Overflow St", "city_name": "Baltimore", "zip": "170112202"}</pre> | (unk) | 26.01% (26.01%) |
|     | <pre>{"first": "Kay", "last": "Ryan", "specialty": "cardiologist", "addr": "Baltimore", "city_name": "Baltimore"}</pre> | (unk) | 13.00% (13.00%) |
|     | <pre>{<br>    "first":"daniel",<br>    "last":"kingkiyanade",<br>    "specialty":"tics",<br>    "legal_name":"",<br>    "addr":"",<br>    "addr2":"",<br>    "city_name":"",<br>    "zip":""<br>}</pre> | (unk) | 8.17% (8.17%) |
|     | <pre>{<br>    "first":"daniel",<br>    "last":"kingkiyanade",<br>    "specialty":"tics",<br>    "legal_name":"",<br>    "addr":"",<br>    "addr2":"",<br>    "city_name":"6000 miami"<br>}</pre> | (unk) | 8.17% (8.17%) |
|     | <pre>{<br>    "city_name": "Baltimore",<br>    "last": "Ryan",<br>   "first": "Kay",<br>   "legal_name": "Virtual Doctor Inc.",<br>   "specialty": "cardiology",<br>  "addr": "123 Main Street",<br>   "zip": "20001"<br> }</pre> | (unk) | 2.18% (2.18%) |
|     | <pre>{<br>    "city_name": "Baltimore",<br>    "last": "Ryan",<br>   "first": "Kay",<br>   "legal_name": "Virtual Doctor Inc.",<br>   "specialty": "cardiology",<br>  "addr": "123 Importance Lane"<br>}</pre> | (unk) | 0.77% (0.77%) |
|     | <pre>{<br>    "city_name": "Baltimore",<br>    "last": "Ryan",<br>   "first": "Kay",<br>   "legal_name": "Virtual Doctor Inc.",<br>   "specialty": "cardiology",<br>   "addr": "7667 Lafayatte Blvd.",<br>   "zip": "30291",<br>   "addr2": "Suite 100"<br>}</pre> | (unk) | 0.75% (0.75%) |
|     | <pre>{<br>    "city_name": "Baltimore",<br>    "last": "Ryan",<br>   "first": "Kay",<br>   "legal_name": "Virtual Doctor Inc.",<br>   "specialty": "cardiology",<br>   "addr": "766 Greater Rd.",<br>   "addr2": "5th floor",<br>   "zip": "890087445"<br>}</pre> | (unk) | 0.71% (0.71%) |
|     | <pre>{<br>    "city_name": "Baltimore",<br>    "last": "Ryan",<br>   "first": "Kay",<br>   "legal_name": "Virtual Doctor Inc.",<br>   "specialty": "cardiology",<br>  "addr": "123 Main St",<br>    "addr2": "apt 320",<br>   "zip": "90210-0509"<br>}</pre> | (unk) | 0.63% (0.63%) |
|     | <pre>{<br>    "city_name": "Baltimore",<br>    "last": "Ryan",<br>   "first": "Kay",<br>   "legal_name": "Virtual Doctor Inc.",<br>   "specialty": "cardiology",<br>  "addr": "123 Fox Ave.",<br>"addr2":"suite 550"<br>}</pre> | (unk) | 0.59% (0.59%) |
|     | <pre>{<br>    "first":"daniel",<br>    "last":"kingkiyanade",<br>    "specialty":"tics",<br>    "legal_name":"",<br>    "addr":"",<br>    "addr2":"",<br>    "city_name":"Baltimore",<br>    "zip":"",<br>    "city_name":"Baltimore"<br>    }</pre> | (unk) | 0.01% (0.01%) |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>{"first": "Kay", "last": "Ryan", "specialty": "appt", "legal_name": "doctor", "city_name": "Baltimore"}</pre> | (unk) | 40.00% (40.00%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "",<br>  "legal_name": "",<br>  "addr": "",<br>  "addr2": "",<br>  "city_name": "Baltimore"<br>}</pre> | (unk) | 33.33% (33.33%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "neurology",<br>    "legal_name": "Green Medicine Inc",<br>    "addr": "256 Overflow St",<br>    "zip": "170112202",<br>    "city_name": "Camp Hill"<br>}</pre> | (unk) | 26.67% (26.67%) |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>{"first": "Kay", "last": "Ryan", "city_name": "Baltimore"}</pre> | (unk) | 87.38% (87.38%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "neurology",<br>  "legal_name": "Happy Brain Services LLC",<br>  "addr": "512 Example Street",<br>  "addr2": "Suite 3600",<br>  "city_name": "Camp Hill"<br>}</pre> | (unk) | 9.60% (9.60%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "neurology",<br>  "legal_name": "HappyBrainServicesLLC",<br>  "addr": "512ExampleStreet",<br>  "addr2": "Suite3600",<br>  "city_name": "CampHill"<br>}</pre> | (unk) | 2.40% (2.40%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "neurology",<br>  "legal_name": "HappyBrainServicesLLC",<br>  "addr": "512ExampleStreet",<br>  "addr2": "Suite3600",<br>  "city_name": "CampRoadham"<br>}</pre> | (unk) | 0.62% (0.62%) |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "pediatrics",<br>    "zip": "12222",<br>    "addr": "760 Upper Pike St",<br>    "addr2": "Apartment 201",<br>    "legal_name": "Glad Heart Care",<br>    "city_name": "Berlin"<br>}</pre> | (unk) | 33.33% (33.33%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "pediatrics",<br>    "legal_name": "null",<br>    "addr": "null",<br>    "addr2": "null",<br>    "zip": "null",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 33.33% (33.33%) |
|     | <pre>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "Baltimore",<br>  "specialty": "Pediatric",<br>  "city_name": "Baltimore",<br>  "zip": "Unknown",<br>  "addr": "Unknown",<br>  "addr2": "Unknown"<br>}</pre> | (unk) | 13.33% (13.33%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "pediatrics",<br>    "zip": "12222",<br>    "addr": "760 Upper Pike St",<br>    "addr2": "Apartment 201",<br>    "legal_name": "Glad Heart Care",<br>    "city_name": "Baltimore"<br>}</pre> | (unk) | 13.33% (13.33%) |
|     | <pre>{<br>    "first": "Kay",<br>    "last": "Ryan",<br>    "specialty": "pediatrics",<br>    "zip": "12222",<br>    "addr": "760 Upper Pike St",<br>    "legal_name": "Children's Hospital of Philadelphia",<br>    "addr2": "Suite 123",<br>    "city_name": "Frankford"<br>}</pre> | (unk) | 6.67% (6.67%) |

### Local inference

In [ ]:
inference_setup = server_model(json_grammar)

In [ ]:
print(inference_setup.sampler.llm._model.batch_size)

In [ ]:
dir(inference_setup.sampler.llm._model)

In [ ]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_vllm(prompt, inference_setup=inference_setup, **server_inference_params)
    responses[sentence] = response

In [ ]:
try:
    del inference_setup
except NameError:
    pass

### Running the model on an arbitrary sentence

In [ ]:
responses = {}
for sentence_datum in tqdm([#{"sentence": "John Smith the podiatrist is terrible, so much worse than John Smith the cardiologist."}, {"sentence": "John Smith works with Jack Smith at the neurology office. He’s terrible at neurology."},
                            {"sentence": "John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!"}]):
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    #break

In [ ]:
def md_html_output_text_for_json(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
            # Assert the response parses properly
            json_parser.parse(output_text)

            #code = extract_code_from_response(output_text)
            code = output_text
            code_with_br = code.replace("\n", "<br>")
            output_with_br = md_html_output_text_for_json(output_text)
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre></pre> '
                f'| <pre>{output_with_br}</pre> '
                f'| {100 * likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))